<h1 align='center'>Testing inside your code
<h2 align='center'>and
<h1 align='center'>Testing the code itself

---
## assert
- A helpful way to **debug** code
- Includes **traceback** (aka stack trace, stack traceback, backtrace) to show you the sequence of calls and associated problems
    - https://realpython.com/python-traceback


- Asserts are not meant to test for expected conditions
    - Security issue: asserts can be turned off globally (in the Python interpreter via 'python -O filename.py'). Therefore, don’t rely on assert expressions to be executed for data validation or data processing.




- syntax: assert =test, 'Error message to display'


In [1]:
## simple example
var_test = 5
assert var_test != 5, 'ERROR MESSAGE'

AssertionError: ERROR MESSAGE

### practical usage

In [4]:
def divide_me(number_1=None, number_2=None):
    return number_1/number_2


divide_me(number_1=1.0, number_2=2.0)

0.5

In [7]:
## Remember the resulting error for later

divide_me(number_1=1.0, number_2=0.0)

AssertionError: Error: you can't divide by 0

In [6]:
def divide_me(number_1=None, number_2=None):
    assert number_2 != 0, "Error: you can't divide by 0"
    
    return number_1/number_2


divide_me(number_1=1.0, number_2=0)

AssertionError: Error: you can't divide by 0

### A word of caution
Let's try to make sure a variable was provided a value
- note the two valid ways to do this

In [29]:
#!/usr/bin/env python
'''
An example for why using assert to test for expected condition is bad.

You can "turn off" asserts by typing "python -O filename.py" and thus
    bypassing the check.

Expectations when running the code:
python assert_example.py -> assert is read and prints out it error
python -O assert_example.py -> assert is not read and prints a standard error
'''

def divide_me(number_1=None, number_2=None):
    assert number_2 != 0, "Error: you can't divide by 0"
    assert number_1 != None, "Error: you did not provide a numerator"
    assert number_2, "Error: you did not provide a denominator"

    return number_1/number_2


divide_me(number_1=1.0)

AssertionError: Error: you did not provide a denominator

However, this is not a good way to do it, because you can get around the assert statement.

Example:

- reads the assert statement (everything seems to work properly)
    - **python assert_example.py**
- the assert statement is not read (i.e. it is bypassed)
    - **python -O assert_example.py**

---
## isinstance
- can check on a variable's type (e.g. int, float, str) 

In [30]:
## 1.0 degree Celcius = 273.15 Kelvins (these are exact numbers)

def celsius_to_kelvin(temperature):
    assert isinstance(temperature, float), 'Error: provided number is not a float'
    return temperature-273


def kelvin_to_celsius(temperature):
    assert temperature >= 0, 'Error: colder than absolute zero!'
    assert isinstance(temperature, float),
    return temperature+273

In [31]:
celsius_to_kelvin(200.5)

-72.5

In [32]:
celsius_to_kelvin(-5)

AssertionError: Error: provided number is not a float

In [1]:
## can also pass it a tuple (via curve brackets)

def celsius_to_kelvin(temperature):
    assert isinstance(temperature, (int, float)),
    return temperature-273

celsius_to_kelvin(-5)

-278

In [33]:
kelvin_to_celsius(-5)

AssertionError: Error: colder than absolute zero!

# Testing for expected conditions
### if statements
    - LBYL: look before you leap, meaning that you check conditions before executing the primary part of the code.

In [31]:
## Example 1

def kelvin_to_celsius(temperature=None):
    if temperature is None:
        print('Error: temperature was not specified.')
        return
    elif temperature <= 0:
        print('Error: temperature is colder than absolute zero!')
        return
    elif isinstance(temperature, (int, float)) == False:
        print('Error: provided termperature is not an integer or float.')
        return
    
    return temperature+273

In [32]:
kelvin_to_celsius()

Error: temperature was not specified.


---
# Exceptions: testing for expected conditions
EAFP: Easier to ask for forgiveness than permission (a pyhtonian idea)


### try-except statments
    - you code will continue when it encounters a problem
    - faster than if statements
    
    - tell your code to try something
    - then tell it what to do if it fails based on a type of exception

Built-in exception types: https://docs.python.org/3/library/exceptions.html

In [87]:
try:
    print(5/0)
except ZeroDivisionError:
    print("You can't have a zero in the denominator.")

You can't have a zero in the denomenator.


### Let's set up an new example for demonstrating try-except

In [67]:
## What would this be using if statements?
## setting the denominator = 0 will results in traceback of the error

print('Type two numbers that you want to be divided.')
print("Type 'q' to quit.")
print()

while True:
    numerator = input('Numerator: ')
    if numerator == 'q':
        break

    denominator = input('Denominator: ')
    if denominator == 'q':
        break
        
    answer = float(numerator)/float(denominator)
    print('Answer for {0}/{1} = {2}\n'.format(numerator, denominator, answer))

Type two numbers that you want to be divided.
Type 'q' to quit.

Numerator: 1
Denominator: 0


ZeroDivisionError: float division by zero

In [68]:
## Improve the if statement a bit more
## setting the denominator = 0 will report error and exit the code

print('Type two numbers that you want to be divided.')
print("Type 'q' to quit.")
print()

while True:
    numerator = input('Numerator: ')
    if numerator == 'q':
        break

    denominator = input('Denominator: ')
    if denominator == 'q':
        break
    if denominator == '0':
        print("You can't have a zero in the denominator.")
        break
        
    answer = float(numerator)/float(denominator)
    print('Answer for {0}/{1} = {2}\n'.format(numerator, denominator, answer))

Type two numbers that you want to be divided.
Type 'q' to quit.

Numerator: 1
Denominator: 0
You can't have a zero in the denominator.


In [69]:
## now do it with a try-except statement
## setting the denominator = 0 will report error and continue the program

print('Type two numbers that you want to be divided.')
print("Type 'q' to quit.")
print()

while True:
    numerator = input('Numerator: ')
    if numerator == 'q':
        break

    denominator = input('Denominator: ')
    if denominator == 'q':
        break
        
    try:
        answer = float(numerator)/float(denominator)
        print('Answer for {0}/{1}: {2}\n'.format(numerator, denominator, answer))
    except ZeroDivisionError:
        print("You can't have a zero in the denominator.")

Type two numbers that you want to be divided.
Type 'q' to quit.

Numerator: 1
Denominator: 0
You can't have a zero in the denominator.
Numerator: q


### One last note
- you can have multiple except conditions

In [25]:
def attempt_float(number):
    try:
        return float(number)
    except (TypeError, ValueError):
        print('Input was not the right type (e.g list for a '
              'single number: TypeError) or correct value (i.e. ValueError).')

In [26]:
attempt_float(0.1)

0.1

In [27]:
attempt_float('0.1') # because float can cast a "proper" string to a float

0.1

In [28]:
attempt_float('something') ## a ValueError

Input was not the right type (e.g list for a single number: TypeError) or correct value (i.e. ValueError).


In [29]:
attempt_float([0.1, 0.2]) ## a TypeError

Input was not the right type (e.g list for a single number: TypeError) or correct value (i.e. ValueError).


In [30]:
## we can simply prove the last is a TypeError
float([0.1, 0.2])

TypeError: float() argument must be a string or a number, not 'list'

##### ---
<h1 align='center'>Test Driven Development
    
<h2 align='center'> a.k.a. Unit Tests

<h3 align='center'> https://docs.python.org/3/library/unittest.html

## Test Driven Development - writing tests before you write your production code
1. Ensures proper and directed functionality of your code
2. Helps you plan your code
3. Reduces errors
4. Helps to ensure code's long life

## Workflow
1. Write failing test
2. Run and ensure failure
3. Write code to pass
4. Run and ensure passing
5. Refactor (i.e. restructure/clean up code without changing it final result)
6. Redo steps 1-5

## Scientific and Data Research
It is CRITICAL that:
1. you get the correct results
2. you make it reproducible as the code becomes bigger (and changes)

In [ ]:
def hello_world():
    return 'hello world'

In [ ]:
import unittest


class MyFirstUniTTests(unittest.TestCase):

    ## Assert Equal but results in a fail
    def test_fail(self):
        self.assertEqual(hello_world(), 'bye world')


    ## Assert Equal
    def test_isequal(self):
        self.assertEqual(hello_world(), 'hello world')


    ## Assert Less Than
    def test_isless(self):
        self.assertLess(5, 10)
        
        
    ## Assert Less Than or Equal
    def test_islessequal(self):
        self.assertLessEqual(10, 10)
        
        
    ## Assert True
    def test_isupper_true(self):
        self.assertTrue('FOO'.isupper())

        
    ## Assert False
    def test_isupper_false(self):
        self.assertFalse('Foo'.isupper())

        
## Normal usage
#if __name__ == '__main__':
#    unittest.main()

## Fir usage in jupyter (due to jupyter's kernel)
if __name__ == '__main__':
    unittest.main(argv=['ignored', '-v'], exit=False)
    

In [38]:
import numpy as np
x = 1.23551
np.around(x, decimals = 3)

1.236

In [41]:
from sigfig import round
signumb=3
round(x, signumb, notation ="sci")

'1.24E0'

In [42]:
"{0:.2g}".format(x)

'1.2'